In [1]:
include("detect.jl")

include("ssd4.jl")
include("config.jl")
include("helpers.jl")
include("detect.jl")
include("visualizations.jl")
include("dataset.jl")
include("transformations.jl")
include("evaluate.jl");
include("detect.jl")
include("dataset.jl")
include("utilities.jl")
include("evaluate.jl")

calculate_mAP (generic function with 1 method)

In [2]:
batch_size = 32
clipart_dt_path = "/home/ckoksal20/Desktop/crossdomainDataset/dt_clipart"
annotation_path_clipart_dt_VOC2007 ="$clipart_dt_path/VOC2007/Annotations"
images_path_clipart_dt_VOC2007 = "$clipart_dt_path/VOC2007/JPEGImages"
main_path_clipart_dt_VOC2007 =  "$clipart_dt_path/VOC2007/ImageSets/Main"

annotation_path_clipart_dt_VOC2012 ="$clipart_dt_path/VOC2012/Annotations"
images_path_clipart_dt_VOC2012 = "$clipart_dt_path/VOC2012/JPEGImages"
main_path_clipart_dt_VOC2012 =  "$clipart_dt_path/VOC2012/ImageSets/Main"


trainval_clipart_dt_VOC2007 = "$main_path_clipart_dt_VOC2007/trainval.txt"
trainval_clipart_dt_VOC2012 = "$main_path_clipart_dt_VOC2012/trainval.txt"





"/home/ckoksal20/Desktop/crossdomainDataset/dt_clipart/VOC2012/ImageSets/Main/trainval.txt"

In [3]:


function initopt!(model::SSD300, startLr = 1e-3,gclip = 0)
    for par in params(model)
        # Authors initialzied learning rates of bias 2x of weights learning rate
        if size(par)[1:2] == (1,1) && size(par)[end] == 1 #bias
            par.opt = Momentum(;lr=2*startLr, gclip=0, gamma = 0.9)
        else
            par.opt = Momentum(;lr=startLr, gclip=0,gamma = 0.9)
        end
    end
end
   
function currentTime()
    dt = now()
    return Dates.format(dt, "yyyy-mm-dd_HH:MM:SS")
end

function finetune!(model::SSD300, train_data, test_data::PascalVOC,epochs
                  ;period::Int=2000, learning_rate_decay = 0.1 )
    
    train_loss = []
    test_loss = []
    iteration = 0
    curTime = currentTime()
    tr_loss = 0
    tst_loss = 0
    println("Training has started ", "length train_data : ",length(train_data), "Expected epochs : ",epochs)
    
    filename="log_ssd300_DT_VOC2007_2012trainval_l1_batch_$batch_size-iteration_$iterations-$curTime.txt"
    for epoch in 1:epochs
        start_epoch_time = now()
        println("Epoch : ", epoch, " has started")
       
        for (x,bounding_boxes,labels,_) in train_data
            println("Current iteration ", iteration)
           
        
            if iteration%period == 0
                tr_loss = 0
                tr_loss = model(train_data) 
                tst_loss = model(test_data)
                println("Iteration ", iteration, " training loss : ",tr_loss, " test_loss : ",tst_loss)

                append!(train_loss,tr_loss )
                append!(test_loss,tst_loss )
            end
            
            momentum!(ssd300,[(x,bounding_boxes,labels)])
            iteration +=1
        end
    end_epoch_time = now()
    elapsed_time = ((end_epoch_time -start_epoch_time).value)/1000/60
        
    println("Elapsed time $elapsed_time min epoch : $epoch\n")    
    open("/home/ckoksal20/Desktop/logfiles/$filename", "a") do f
        write(f, "Elapsed time $elapsed_time min epoch : $epoch\n")
        #write(f,"Iteration: $iteration, training loss : $tr_loss, test_loss : $tst_loss \n")
    end
    end
        
    return 0:period:iteration, train_loss, test_loss
end

batch_size = 4

clipart_train = PascalVOC([trainval_clipart_dt_VOC2007,trainval_clipart_dt_VOC2012], 
    [annotation_path_clipart_dt_VOC2007,annotation_path_clipart_dt_VOC2012],
    
    [images_path_clipart_dt_VOC2007,images_path_clipart_dt_VOC2012],
    "TEST",dtype = atype, batchsize =batch_size, multi_dataset=true,shuffle = true)



clipart_path = "/home/ckoksal20/Desktop/crossdomainDataset/clipart"
annotation_path_clipart= "$clipart_path/Annotations"
images_path_clipart= "$clipart_path/JPEGImages"
main_path_clipart =  "$clipart_path/ImageSets/Main"
test_clipart = "$main_path_clipart/test.txt"
clipart_test = PascalVOC(test_clipart, annotation_path_clipart
    , images_path_clipart,"TEST",dtype = atype, batchsize =batch_size)


ssd300 = SSD300(pretrained = true)
initopt!(ssd300, 1e-6)

before_ft_APs, before_ft_mAP  = evaluate(clipart_test,ssd300)



iterations = 1000
epochs = round(iterations/length(clipart_train))
epochs=1

iters, trnloss, tstloss = finetune!(ssd300,clipart_train,clipart_test,epochs)
after_finetune_ap,after_finetune_Map = evaluate(watercolor_test,ssd300)



Pretrained constructor is called
 Iteration : 1
 Iteration : 2
 Iteration : 3
 Iteration : 4
 Iteration : 5
 Iteration : 6
 Iteration : 7
 Iteration : 8
 Iteration : 9
 Iteration : 10
 Iteration : 11
 Iteration : 12
 Iteration : 13
 Iteration : 14
 Iteration : 15
 Iteration : 16
 Iteration : 17
 Iteration : 18
 Iteration : 19
 Iteration : 20
 Iteration : 21
 Iteration : 22
 Iteration : 23
 Iteration : 24
 Iteration : 25
 Iteration : 26
 Iteration : 27
 Iteration : 28
 Iteration : 29
 Iteration : 30
 Iteration : 31
 Iteration : 32
 Iteration : 33
 Iteration : 34
 Iteration : 35
 Iteration : 36
 Iteration : 37
 Iteration : 38
 Iteration : 39
 Iteration : 40
 Iteration : 41
 Iteration : 42
 Iteration : 43
 Iteration : 44
 Iteration : 45
 Iteration : 46
 Iteration : 47
 Iteration : 48
 Iteration : 49
 Iteration : 50
 Iteration : 51
 Iteration : 52
 Iteration : 53
 Iteration : 54
 Iteration : 55
 Iteration : 56
 Iteration : 57
 Iteration : 58
 Iteration : 59
 Iteration : 60
 Iteration : 61


Current iteration 237
Current iteration 238
Current iteration 239
Current iteration 240
Current iteration 241
Current iteration 242
Current iteration 243
Current iteration 244
Current iteration 245
Current iteration 246
Current iteration 247
Current iteration 248
Current iteration 249
Current iteration 250
Current iteration 251
Current iteration 252
Current iteration 253
Current iteration 254
Current iteration 255
Current iteration 256
Current iteration 257
Current iteration 258
Current iteration 259
Current iteration 260
Current iteration 261
Current iteration 262
Current iteration 263
Current iteration 264
Current iteration 265
Current iteration 266
Current iteration 267
Current iteration 268
Current iteration 269
Current iteration 270
Current iteration 271
Current iteration 272
Current iteration 273
Current iteration 274
Current iteration 275
Current iteration 276
Current iteration 277
Current iteration 278
Current iteration 279
Current iteration 280
Current iteration 281
Current it

Current iteration 609
Current iteration 610
Current iteration 611
Current iteration 612
Current iteration 613
Current iteration 614
Current iteration 615
Current iteration 616
Current iteration 617
Current iteration 618
Current iteration 619
Current iteration 620
Current iteration 621
Current iteration 622
Current iteration 623
Current iteration 624
Current iteration 625
Current iteration 626
Current iteration 627
Current iteration 628
Current iteration 629
Current iteration 630
Current iteration 631
Current iteration 632
Current iteration 633
Current iteration 634
Current iteration 635
Current iteration 636
Current iteration 637
Current iteration 638
Current iteration 639
Current iteration 640
Current iteration 641
Current iteration 642
Current iteration 643
Current iteration 644
Current iteration 645
Current iteration 646
Current iteration 647
Current iteration 648
Current iteration 649
Current iteration 650
Current iteration 651
Current iteration 652
Current iteration 653
Current it

Current iteration 981
Current iteration 982
Current iteration 983
Current iteration 984
Current iteration 985
Current iteration 986
Current iteration 987
Current iteration 988
Current iteration 989
Current iteration 990
Current iteration 991
Current iteration 992
Current iteration 993
Current iteration 994
Current iteration 995
Current iteration 996
Current iteration 997
Current iteration 998
Current iteration 999
Current iteration 1000
Current iteration 1001
Current iteration 1002
Current iteration 1003
Current iteration 1004
Current iteration 1005
Current iteration 1006
Current iteration 1007
Current iteration 1008
Current iteration 1009
Current iteration 1010
Current iteration 1011
Current iteration 1012
Current iteration 1013
Current iteration 1014
Current iteration 1015
Current iteration 1016
Current iteration 1017
Current iteration 1018
Current iteration 1019
Current iteration 1020
Current iteration 1021
Current iteration 1022
Current iteration 1023
Current iteration 1024
Current

Current iteration 1339
Current iteration 1340
Current iteration 1341
Current iteration 1342
Current iteration 1343
Current iteration 1344
Current iteration 1345
Current iteration 1346
Current iteration 1347
Current iteration 1348
Current iteration 1349
Current iteration 1350
Current iteration 1351
Current iteration 1352
Current iteration 1353
Current iteration 1354
Current iteration 1355
Current iteration 1356
Current iteration 1357
Current iteration 1358
Current iteration 1359
Current iteration 1360
Current iteration 1361
Current iteration 1362
Current iteration 1363
Current iteration 1364
Current iteration 1365
Current iteration 1366
Current iteration 1367
Current iteration 1368
Current iteration 1369
Current iteration 1370
Current iteration 1371
Current iteration 1372
Current iteration 1373
Current iteration 1374
Current iteration 1375
Current iteration 1376
Current iteration 1377
Current iteration 1378
Current iteration 1379
Current iteration 1380
Current iteration 1381
Current ite

...

Current iteration 2758
Current iteration 2759
Current iteration 2760
Current iteration 2761
Current iteration 2762
Current iteration 2763
Current iteration 2764
Current iteration 2765
Current iteration 2766
Current iteration 2767
Current iteration 2768
Current iteration 2769
Current iteration 2770
Current iteration 2771
Current iteration 2772
Current iteration 2773
Current iteration 2774
Current iteration 2775
Current iteration 2776
Current iteration 2777
Current iteration 2778
Current iteration 2779
Current iteration 2780
Current iteration 2781
Current iteration 2782
Current iteration 2783
Current iteration 2784
Current iteration 2785
Current iteration 2786
Current iteration 2787
Current iteration 2788
Current iteration 2789
Current iteration 2790
Current iteration 2791
Current iteration 2792
Current iteration 2793
Current iteration 2794
Current iteration 2795
Current iteration 2796
Current iteration 2797
Current iteration 2798
Current iteration 2799
Current iteration 2800
Current ite

Current iteration 3114
Current iteration 3115
Current iteration 3116
Current iteration 3117
Current iteration 3118
Current iteration 3119
Current iteration 3120
Current iteration 3121
Current iteration 3122
Current iteration 3123
Current iteration 3124
Current iteration 3125
Current iteration 3126
Current iteration 3127
Current iteration 3128
Current iteration 3129
Current iteration 3130
Current iteration 3131
Current iteration 3132
Current iteration 3133
Current iteration 3134
Current iteration 3135
Current iteration 3136
Current iteration 3137
Current iteration 3138
Current iteration 3139
Current iteration 3140
Current iteration 3141
Current iteration 3142
Current iteration 3143
Current iteration 3144
Current iteration 3145
Current iteration 3146
Current iteration 3147
Current iteration 3148
Current iteration 3149
Current iteration 3150
Current iteration 3151
Current iteration 3152
Current iteration 3153
Current iteration 3154
Current iteration 3155
Current iteration 3156
Current ite

Current iteration 3470
Current iteration 3471
Current iteration 3472
Current iteration 3473
Current iteration 3474
Current iteration 3475
Current iteration 3476
Current iteration 3477
Current iteration 3478
Current iteration 3479
Current iteration 3480
Current iteration 3481
Current iteration 3482
Current iteration 3483
Current iteration 3484
Current iteration 3485
Current iteration 3486
Current iteration 3487
Current iteration 3488
Current iteration 3489
Current iteration 3490
Current iteration 3491
Current iteration 3492
Current iteration 3493
Current iteration 3494
Current iteration 3495
Current iteration 3496
Current iteration 3497
Current iteration 3498
Current iteration 3499
Current iteration 3500
Current iteration 3501
Current iteration 3502
Current iteration 3503
Current iteration 3504
Current iteration 3505
Current iteration 3506
Current iteration 3507
Current iteration 3508
Current iteration 3509
Current iteration 3510
Current iteration 3511
Current iteration 3512
Current ite

Current iteration 3826
Current iteration 3827
Current iteration 3828
Current iteration 3829
Current iteration 3830
Current iteration 3831
Current iteration 3832
Current iteration 3833
Current iteration 3834
Current iteration 3835
Current iteration 3836
Current iteration 3837
Current iteration 3838
Current iteration 3839
Current iteration 3840
Current iteration 3841
Current iteration 3842
Current iteration 3843
Current iteration 3844
Current iteration 3845
Current iteration 3846
Current iteration 3847
Current iteration 3848
Current iteration 3849
Current iteration 3850
Current iteration 3851
Current iteration 3852
Current iteration 3853
Current iteration 3854
Current iteration 3855
Current iteration 3856
Current iteration 3857
Current iteration 3858
Current iteration 3859
Current iteration 3860
Current iteration 3861
Current iteration 3862
Current iteration 3863
Current iteration 3864
Current iteration 3865
Current iteration 3866
Current iteration 3867
Current iteration 3868
Current ite

LoadError: UndefVarError: watercolor_test not defined

In [4]:
ssd300

SSD300(VGG16(Chain(Any[Conv(P(KnetArray{Float32,4}(3,3,3,64)), P(KnetArray{Float32,4}(1,1,64,1)), Knet.Ops20.relu, 1, 1, 1, 1, 0, 1, KnetArray{Float32,N} where N, true), Conv(P(KnetArray{Float32,4}(3,3,64,64)), P(KnetArray{Float32,4}(1,1,64,1)), Knet.Ops20.relu, 1, 1, 2, 2, 0, 1, KnetArray{Float32,N} where N, true), Conv(P(KnetArray{Float32,4}(3,3,64,128)), P(KnetArray{Float32,4}(1,1,128,1)), Knet.Ops20.relu, 1, 1, 1, 1, 0, 1, KnetArray{Float32,N} where N, true), Conv(P(KnetArray{Float32,4}(3,3,128,128)), P(KnetArray{Float32,4}(1,1,128,1)), Knet.Ops20.relu, 1, 1, 2, 2, 0, 1, KnetArray{Float32,N} where N, true), Conv(P(KnetArray{Float32,4}(3,3,128,256)), P(KnetArray{Float32,4}(1,1,256,1)), Knet.Ops20.relu, 1, 1, 1, 1, 0, 1, KnetArray{Float32,N} where N, true), Conv(P(KnetArray{Float32,4}(3,3,256,256)), P(KnetArray{Float32,4}(1,1,256,1)), Knet.Ops20.relu, 1, 1, 1, 1, 0, 1, KnetArray{Float32,N} where N, true), Conv(P(KnetArray{Float32,4}(3,3,256,256)), P(KnetArray{Float32,4}(1,1,256,1)), 

In [ ]:
function subsetMAP(mAP)
    subset_elements = ["bird","bicycle","cat","car","dog","person"]
    num_subsetElements = size(subset_elements,1)
    total_map = 0
    for (k,v) in mAP
        if k in subset_elements
            total_map += v
        end
        
    end
    return total_map / num_subsetElements
end

In [ ]:
subsetMAP(before_ft_mAP)

In [ ]:
before_ft_mAP

In [ ]:

before_ft_APs, before_ft_mAP  = evaluate(clipart_test,ssd300)


In [ ]:
after_finetune_ap,after_finetune_Map = evaluate(clipart_test,ssd300)


In [ ]:
before_ft_APs

In [ ]:
before_ft_mAP

In [ ]:
after_finetune_ap

In [ ]:
after_finetune_Map